In [ ]:
OPEN_AI_API_KEY = "API_KEY"

In [2]:
from langchain_community.document_loaders import YoutubeLoader

docs = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=pbAd8O1Lvm4", add_video_info=True
).load()

docs[0].metadata

{'source': 'pbAd8O1Lvm4',
 'title': 'Self-reflective RAG with LangGraph: Self-RAG and CRAG',
 'description': 'Unknown',
 'view_count': 17654,
 'thumbnail_url': 'https://i.ytimg.com/vi/pbAd8O1Lvm4/hq720.jpg',
 'publish_date': '2024-02-07 00:00:00',
 'length': 1058,
 'author': 'LangChain'}

In [3]:
import datetime
from typing import Literal, Optional, Tuple
from langchain_core.pydantic_v1 import BaseModel, Field

In [4]:
class TutorialSearch(BaseModel):
    company_name: Optional[str] = Field(
        None, 
        description="String described as a company name. Only use if explicitly specified."
    ) 

    def pretty_print(self) -> None:
        for field in self.__fields__:
            if getattr(self, field) is not None and getattr(self, field) != getattr(self.__fields__[field], "default", None):
                print(f"{field}: {getattr(self, field)}")

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

system = """ You are an expert at converting user questions into database queries. 
    You have access to a database of annual reports the top companies. 
    Given a question, return a database query optimized to retrieve the most relevant results. 
    
    If there are acronymns or words you are not familiar with, do not try to rephrase them. """

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

llm = ChatOpenAI(model = "gpt-3.5-turbo-0125", temperature=0, openai_api_key = OPEN_AI_API_KEY)
structured_llm = llm.with_structured_output(TutorialSearch)
query_analyzer = prompt | structured_llm

In [6]:
query_analyzer.invoke({"question": "Total assets of AIS."})

TutorialSearch(author_search='AIS')

In [3]:
lowercase_question = "Total assets of AIS"

if "ais" in lowercase_question.lower():
    print("true")


true


In [4]:
fruits = ["apple", "banana", "orange"]
string = "apple banana apple orange apple banana orange apple"

counts = {fruit: string.count(fruit) for fruit in fruits}
print(counts)


{'apple': 4, 'banana': 2, 'orange': 2}


In [7]:
sum([string.count(fruit) for fruit in fruits])

8

In [2]:
import re
query = "What is total assets of AIS in 2023?"
re.sub(r'[^A-Za-z0-9 ]+', '', query)

'What is total assets of AIS in 2023'